In [1]:
import numpy as np
import copy
from scipy import misc
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, CyclicLR, ReduceLROnPlateau, LinearLR, ExponentialLR
import random
from torch.autograd.functional import jacobian, hessian
# import AUTOGRAD.FUNCTIONAL.JACOBIAN as jacobian
import time
import math

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
mpl.rcParams['legend.fontsize'] = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
!python -V

Python 3.11.7


# Parameters of the PDEs

In [3]:
dim=9
params={'dim':dim,
          'kappa':torch.tensor([0.,1.,0.8,0.6,0.4,0.5,0.3,0.2,0.1,0.7]).to(device), # The first kappa=0 because the drift of wealth process is zero
          'theta':torch.tensor([0.,0.1,0.2,0.3,0.4,0.5,0.4,0.3,0.2,0.1]).to(device),
          'nu':torch.tensor([0.02,0.015,0.11,0.12,0.01,0.013,0.14,0.14,0.01]).to(device),
          'lb':torch.tensor([0.,0.15,0.11,0.12,0.13,0.15,0.11,0.12,0.13,0.15]).to(device),   
          'rho':torch.tensor([0.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]).to(device),
          'eta':torch.tensor([1]).to(device),
          'T': torch.tensor([1]).to(device)
        }
params0={'dim':dim,
          'kappa':torch.tensor([0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]).to(device), # The first kappa=0 because the drift of wealth process is zero
          'theta':torch.tensor([0.,0.1,0.2,0.3,0.4,0.5,0.4,0.3,0.2,0.1]).to(device),
          'nu':torch.tensor([0.02,0.015,0.11,0.12,0.01,0.013,0.14,0.14,0.01]).to(device),
          'lb':torch.tensor([0.,0.15,0.11,0.12,0.13,0.15,0.11,0.12,0.13,0.15]).to(device),   
          'rho':torch.tensor([0.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]).to(device),
          'eta':torch.tensor([1.0]).to(device),
          'T': torch.tensor([1.]).to(device)
        }

# Autograd for gradient and Hessian

## Gradient

In [4]:

def der1(x,f):  #output= [M,D,1], input: x=[M,D,1]
  Du=torch.zeros(x.shape[0],dim).to(device)
  xin=x.clone().detach()
  xin.requires_grad=True
  # print('xin shape', xin.shape)
  u=f(xin)
  # print(u)
  # print('u/f(xin) shape', u.shape)
  Du=torch.autograd.grad(outputs=[u],inputs=[xin],grad_outputs=torch.ones_like(u),
                          allow_unused=True,retain_graph=True,create_graph=True)[0].unsqueeze(2)
  # print('Du shape before reshaped', Du.shape)
  Du=torch.reshape(Du,(Du.shape[0],dim,1))
  # print('Du shape after reshaped', Du.shape)
  return Du



In [5]:


def grad_model(t, x,f_): #output= [M,D,1], #input: x=[M,D], t=[M,1], xt= [M,D+1]
    # hess_temp=torch.zeros(x.shape[0],dim,dim).to(device)
    Du=torch.zeros(x.shape[0],dim).to(device)

    xin=x.clone().detach()
    xin.requires_grad=True
    # print("xin_shape", xin.shape)
    tin=t.clone().detach()
    # print("tin shape", tin.shape)
    xt_in=torch.cat((xin,tin),1)
    # print("xt_in shape", xt_in.shape)

    # xt_in.requires_grad=True
    u=f_(xt_in)
    # print("u shape: ",u.shape)
    Du=torch.autograd.grad(outputs=[u],inputs=[xin],grad_outputs=torch.ones_like(u),
                           allow_unused=True,retain_graph=True,create_graph=True)[0].unsqueeze(2)

    Du = torch.reshape(Du,(Du.shape[0],dim,1))
    # print("Du after reshape:",Du.shape)

    return Du



##  First derivative

In [6]:


def grad_model(t, x,f_): #output= [M,D,1], #input: x=[M,D], t=[M,1], xt= [M,D+1]
    # hess_temp=torch.zeros(x.shape[0],dim,dim).to(device)
    Du=torch.zeros(x.shape[0],dim).to(device)

    xin=x.clone().detach()
    xin.requires_grad=True
    # print("xin_shape", xin.shape)
    tin=t.clone().detach()
    # print("tin shape", tin.shape)
    xt_in=torch.cat((xin,tin),1)
    # print("xt_in shape", xt_in.shape)

    # xt_in.requires_grad=True
    u=f_(xt_in)
    # print("u shape: ",u.shape)
    Du=torch.autograd.grad(outputs=[u],inputs=[xin],grad_outputs=torch.ones_like(u),
                           allow_unused=True,retain_graph=True,create_graph=True)[0].unsqueeze(2)

    Du = torch.reshape(Du,(Du.shape[0],dim,1))
    # print("Du after reshape:",Du.shape)

    return Du



## Gradient

In [7]:

def der1(x,f):  #output= [M,D,1], input: x=[M,D,1]
  Du=torch.zeros(x.shape[0],dim).to(device)
  xin=x.clone().detach()
  xin.requires_grad=True
  # print('xin shape', xin.shape)
  u=f(xin)
  # print(u)
  # print('u/f(xin) shape', u.shape)
  Du=torch.autograd.grad(outputs=[u],inputs=[xin],grad_outputs=torch.ones_like(u),
                          allow_unused=True,retain_graph=True,create_graph=True)[0].unsqueeze(2)
  # print('Du shape before reshaped', Du.shape)
  Du=torch.reshape(Du,(Du.shape[0],dim,1))
  # print('Du shape after reshaped', Du.shape)
  return Du



In [8]:

def grad_fn2(x,z,a,f,xx=False,r=False,p=False,alpha=False):  #output= [M,D,1], input: x=[M,D,1] z=[M,D,1], a=[M,D,D]
  Du=torch.zeros(x.shape[0],dim).to(device)
  xin=x.clone().detach()
  xin.requires_grad=True
  # print('xin shape', xin.shape)
  zin=z.clone().detach()
  zin.requires_grad=True
  # print('zin shape', zin.shape)
  ain=a.clone().detach()
  ain.requires_grad=True
  # print('ain shape', ain.shape)
  # ain=torch.reshape(ain,(num_sample,dim*dim,1))
  u=f(xin,zin,ain)
  # print(u)
  # print('u/f(xin) shape', u.shape)
  if xx==True:
    Du=torch.autograd.grad(outputs=[u],inputs=[xin],grad_outputs=torch.ones_like(u),
                            allow_unused=True,retain_graph=True,create_graph=True)[0]
    # print('Du shape before reshaped', Du.shape)
    Du=torch.reshape(Du,(Du.shape[0],dim,1))
    # print('Du shape after reshaped', Du.shape)
  if r==True:
    Du=torch.autograd.grad(outputs=[u],inputs=[rin],grad_outputs=torch.ones_like(u),
                            allow_unused=True,retain_graph=True,create_graph=True)[0]
    # print('Du shape before reshaped', Du.shape)
    Du=torch.reshape(Du,(Du.shape[0],1,1))
    # Du=torch.reshape(Du,(Du.shape[0],dim,dim))
    # print('Du shape after reshaped', Du.shape)
  if p==True:
    Du=torch.autograd.grad(outputs=[u],inputs=[zin],grad_outputs=torch.ones_like(u),
                            allow_unused=True,retain_graph=True,create_graph=True)[0]
    # print('Du shape before reshaped', Du.shape)
    Du=torch.reshape(Du,(Du.shape[0],dim,1))
    # print('Du shape after reshaped', Du.shape)
  if alpha==True:
    Du=torch.autograd.grad(outputs=[u],inputs=[ain],grad_outputs=torch.ones_like(u),
                            allow_unused=True,retain_graph=True,create_graph=True)[0]
    # print('Du shape before reshaped', Du.shape)
    Du=torch.reshape(Du,(Du.shape[0],dim,dim))
    # print('Du shape after reshaped', Du.shape)

  return Du




## Jacobian

In [9]:
from torch.autograd.functional import jacobian    

In [10]:
    
class sample_Znet(nn.Module): #input [M,D+1]   #output [M,1]
    def __init__(self):
        super(sample_Znet, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(dim+1, 10),
            # nn.BatchNorm1d(num_features=20),
            nn.Tanh(),
            nn.Linear(10,dim),
        )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits#.reshape([dim,dim])       
dv = sample_Znet()
def test_fnc(x):
    return x[:,1:]*x[:,1:] + torch.sin(torch.tensor([2])*torch.pi*x[:,1:])

In [11]:
test_fnc(input)

TypeError: 'method' object is not subscriptable

In [ ]:
input = x_tmp
print(input.shape,dim)
test_fnc(input)

In [ ]:
torch.sin(2*torch.pi*input)

In [ ]:
output = jacobian(test_fnc, input)
print(input.shape, test_fnc(input).shape,output.shape)
for i in range(input.shape[0]):
    if i == 0:
        jac = output[0,:,0,1:].unsqueeze(0)# output[0,:,0,1:] eliminates the derivative wrt the first compinent, i.e., time derivative
        # print(output[i,:,i,:])
    else:
        # print(output[i,:,i,:].unsqueeze(0))
        jac = torch.cat((jac,output[i,:,i,1:].unsqueeze(0)))
print("Jacobians Tensor:\n", jac)

In [ ]:
output

In [ ]:
output[0,:,0,1:]

Jacobian of a function from $\mathbb{R}^d$ to $\mathbb{R}^d$, excludes time derivative

In [ ]:
def sqr_jacobian(func,x):#Jacobian of a function from R^d to R^d
    xin=x.clone().detach()
    xin.requires_grad=True
    out = jacobian(func, xin)
    for i in range(input.shape[0]):
        if i == 0:
            jac = out[i,:,i,1:].unsqueeze(0)
            # print(output[i,:,i,:])
        else:
            # print(output[i,:,i,:].unsqueeze(0))
            jac = torch.cat((jac,out[i,:,i,1:].unsqueeze(0)))
    return jac
    

In [ ]:
sqr_jacobian(test_fnc, input)

## Hessian

In [ ]:
def hessian_model(t,x,model):
    hess_temp=torch.zeros(x.shape[0],dim,dim).to(device)
    Du=torch.zeros(x.shape[0],dim).to(device)

    xin=x.clone().detach()
    xin.requires_grad=True
    # print("xin_shape", xin.shape)
    tin=t.clone().detach()
    # print("tin shape", tin.shape)
    xt_in=torch.cat((xin,tin),1)
    # print("xt_in shape", xt_in.shape)
    Du = model(xt_in)
    hess_temp= torch.cat([ torch.autograd.grad(outputs=[Du[:,i]],inputs=[xin],grad_outputs=torch.ones_like(Du[:,i]),
                           allow_unused=True,retain_graph=True,create_graph=True)[0] for i in range(dim)],1)

    # print("D2u shape:",hess_temp.shape)

    hess_temp=torch.reshape(hess_temp,(hess_temp.shape[0],dim,dim))

    # print(hess_temp)
    return hess_temp



In [ ]:

def hessian1(x,model2z):  #x=[M,D,1]  output= [M,D,1]
    hess_temp=torch.zeros(x.shape[0],dim,dim).to(device)
    # print("D2u shape:",hess_temp.shape)

    Du=torch.zeros(x.shape[0],dim).to(device)
    xin=x.clone().detach()
    # print('this is xin')
    # print(xin)
    xin.requires_grad=True
    Du = model2z(xin.squeeze(2))
    # print('this is Du')
    # print(Du)
    # print('this is Du shape')
    # print(Du.shape)
    # print('this is Du 1')
    # print(Du[:,1,:])
    hess_temp= torch.cat([ torch.autograd.grad(outputs=[Du[:,i,:]],inputs=[xin],grad_outputs=torch.ones_like(Du[:,i,:]),
                           allow_unused=True,retain_graph=True,create_graph=True)[0] for i in range(dim)],1)

    # print("D2u shape:",hess_temp.shape)

    hess_temp=torch.reshape(hess_temp,(hess_temp.shape[0],dim,dim))

    # print(hess_temp)
    return hess_temp




## First and second derivative together

In [ ]:
def Grad_Hess(x,v): #output= [M,D,D], #input: x=[M,D], t=[M,1], xt= [M,D+1]
    hess_temp=torch.zeros(x.shape[0],dim,dim).to(device)
    Du=torch.zeros(x.shape[0],dim).to(device)
    xin=x.clone().detach()
    xin.requires_grad=True
    u=v(xin)
    Du=torch.autograd.grad(outputs=[u],inputs=[xin],grad_outputs=torch.ones_like(u),
                           allow_unused=True,retain_graph=True,create_graph=True)[0].unsqueeze(2)
    hess_temp= torch.cat([ torch.autograd.grad(outputs=[Du[:,i,:]],inputs=[xin],grad_outputs=torch.ones_like(Du[:,i,:]),
                           allow_unused=True,retain_graph=True,create_graph=True)[0] for i in range(dim)],1)
    Du = torch.reshape(Du,(Du.shape[0],dim,1))
    hess_temp=torch.reshape(hess_temp,(hess_temp.shape[0],dim,dim))
    return Du, hess_temp



In [ ]:
x_tmp = torch.tensor([[0.6253, 0.2105, 0.6508, 0.8284, 0.4725, 0.6971, 0.4351, 0.8187, 0.4427],
        [0.8233, 0.8866, 0.3864, 0.8186, 0.0874, 0.6627, 0.9699, 0.2465, 0.6860]])
class sample_Ynet(nn.Module): #input [M,D+1]   #output [M,1]
    def __init__(self):
        super(sample_Ynet, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(dim, 10),
            nn.Sigmoid(),
            nn.Linear(10,1),
        )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits#.reshape([dim,dim])  
v = sample_Ynet()
Grad_Hess(x_tmp[:,0:dim],v)[1]

# Define NNs and coefficient functions

## Neural nets


### NNs for the solution and derivative

In [ ]:
class Ynet(nn.Module): #input [M,D+1]   #output [M,1]
    def __init__(self):
        super(Ynet, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(dim, 10),
            # nn.BatchNorm1d(num_features=10),
            nn.Tanh(),
            nn.Linear(10,1),
        )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits#.reshape([dim,dim])  
    
class Znet(nn.Module): #input [M,D+1]   #output [M,1]
    def __init__(self):
        super(Znet, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(dim+1, 10),
            # nn.BatchNorm1d(num_features=20),
            nn.Tanh(),
            nn.Linear(10,dim),
        )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits#.reshape([dim,dim])         

### 1d NN for the diffusion coefficient of the wealth process

In [ ]:
class sigma1D(nn.Module): #input [M,D+1]   #output [M,1]
    def __init__(self,dim):
        super(sigma1D, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(dim, 4),
            nn.BatchNorm1d(num_features=4),
            nn.Tanh(),
            nn.Linear(4,1),
        )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits#.reshape([dim,dim])
    
class sigma(nn.Module): #input [M,D+1]   #output [M,1]
    def __init__(self,dim):
        super(sigma, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(dim, 4),
            nn.BatchNorm1d(num_features=4),
            nn.Tanh(),
            nn.Linear(4,dim*dim),
        )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits#.reshape([dim,dim])    


## Coefficients

### Generic class that defines the coefficients of the PDEs

In [ ]:
'''Implements the coefficeints based on the parameters '''
class coefficient(object):
    def __init__(self,params): #### out_shape =([M]) |  input:  x_shape=[M,D,1],  z shape = [M,D,1],a_shape= [M,D,D]  #This is for rho=0
        self.dim=params['dim']#2
        self.nu = params['nu']
        self.kappa = params['kappa']
        self.theta = params['theta']
        self.eta = params['eta']
        self.lb = params['lb']
        
    

### Diffuion coefficient

In [ ]:
'''constant diffution coefficient'''  

class constant_diff(coefficient):
    '''This class is a constant diffusion coefficient which is the optimal diffusion'''
    def __init__(self,params):
        super(constant_diff, self).__init__(params)
    def __call__(self,x):
        tmp = x.shape[0]
        s = torch.sqrt(torch.pow(self.lb,2).sum())/self.eta
        return torch.diag(torch.cat((s,self.nu[0:self.dim-1]),axis=0)).repeat(tmp,1,1)

'''Random diffusion coefficient for wealth process with diffusion of volatility processes all constant'''   
class random_diff(coefficient):
    def __init__(self,params):
        super(random_diff, self).__init__(params)
    def __call__(self,x):
        tmp = x.shape[0]
        return torch.diag(torch.cat((torch.rand(tmp,1),self.nu[0:self.dim-1].repeat(tmp,1)),axis=0))
    
'''NN diffusion coefficient for wealth process  with diffusion of volatility processes all constant'''
class NN_diff_1D(coefficient):
    def __init__(self,params,**kwargs):
        super(NN_diff_1D, self).__init__(params)
        if kwargs:
            if 'diff' in kwargs.keys():
                self.s = kwargs['diff']
            else:
                self.s = sigma1D(self.dim)
                print("Warning: since the coefficient in not created using diff = diffusion(params), the diffusion coefficient is set to a neural net in first variable.")
            if 'update' in kwargs.keys():
                self.update = kwargs['update']
            else:
                self.update = lambda x: torch.zeros(x.shape[0],1)
                print("Warning: since the coefficient in not created using update = function, the update function is set to zero.")
        else:
            self.s = sigma1D(self.dim)#Define an update function only for this coefficient to change its value.
            self.update = lambda x: torch.zeros(x.shape[0],1) 
    def __call__(self,x):
        tmp = self.s(x)+self.update(x)
        shp = x.shape[0]
        return torch.diag_embed(torch.cat((tmp,self.nu[0:self.dim-1].repeat(shp,1)),axis=1))

# class updated(coefficient):
#     def __init__(self,params,coef,adjust):
#         self.adjust = adjust
#         self.coef = coef
#         super(update, self).__init__(params)
#     def __call__(self,x):
#         return self.coef(x) + self.adjust(x)    
    
'''NN diffusion for all components'''    
class NN_diff(coefficient):
    def __init__(self,params):
        super(NN_diff, self).__init__(params)
        self.s = sigma(self.dim)
    def __call__(self,x):
        return self.s0(x)
    


In [ ]:
s = sigma1D(2)
u = sigma1D(2)


In [ ]:
t = NN_diff_1D(params,diff = s, update = u)

In [ ]:
t(input[:,1:])

### Zero drift 

In [ ]:
'''Zero drift coefficient for all components'''  
class zero_drift(coefficient):
    def __init__(self,params):
        super(zero_drift, self).__init__(params)
    def __call__(self,x):
        return torch.zeros(x.shape[0],x.shape[1]-1)

### Drift for the semilinear

In [ ]:
'''Drift of semilinear eqn: first component drift=0, others are OU'''
    
class OU_drift_semi(coefficient):
    def __init__(self,params):
        super(OU_drift_semi, self).__init__(params)
    def __call__(self,x):
        num_samples = x.shape[0]
        output = torch.zeros(num_samples,dim)
        for i in range(0,self.dim):
            output[:,i] = self.kappa[i]*(self.theta[i] - x[:,i])
        return output
 
 

### Drift for linear equation

In [ ]:
'''Drift of linear eqn: first component drift =|lb|*sqn(v_e)sigma_{00}(x), others are OU'''

class OU_drift_lin(coefficient):
    def __init__(self,params,v,sigma):
        self.p = v
        self.sigma = sigma
        super(OU_drift_lin, self).__init__(params)
    def __call__(self,x):
        num_samples = x.shape[0]
        q = self.p(x)[:,0]
        output = torch.zeros(num_samples,dim)
        output[:,0] = torch.sqrt(torch.pow(self.lb,2).sum())*torch.sgn(q)*self.sigma(x)[:,0,0]
        for i in range(1,self.dim):
            output[:,i] = self.kappa[i]*(self.theta[i] - x[:,i])
        return output        
    


### Driver of the semilinear BSDE

In [ ]:
class f_driver(coefficient):
    def __init__(self,params): #### out_shape =([M]) |  input:  x_shape=[M,D,1],  z shape = [M,D,1],a_shape= [M,D,D]  #This is for rho=0
        super(f_driver, self).__init__(params)
    def __call__(self,x,z,a):
        output=torch.zeros([x.shape[0]]).to(device)
        for i in range(1,self.dim):
            output = output - self.kappa[i]*(self.theta[i]-x[:,i,0])*z[:,i,0]
        # print(result)
        return output-torch.sqrt(torch.sum(torch.square(self.lb)))*torch.abs(z[:,0,0])*torch.abs(a)


### Source

In [ ]:
class zero_source(coefficient):
    def __init__(self,params):
        super(zero_source, self).__init__(params)
    def __call__(self,x):
        return torch.zeros(x.shape[0],1)    
class ell_source(coefficient):
    def __init__(self,params,v,sigma,alpha_n):
        self.p = v
        self.sigma = sigma
        self.alpha = alpha_n
        super(ell_source, self).__init__(params)
    def direction(self,x):
        num_samples = x.shape[0]        
        q = self.p(x)[:,0]
        gamma = sqr_jacobian(self.p,x)
        x__ = torch.zeros(num_samples,dim,dim)
        x__[:,0,0] = torch.sqrt(torch.pow(self.lb,2).sum())*torch.abs(q)
        return x__+torch.transpose(self.sigma(x),1,2)*gamma
    def __call__(self,x):
        return  self.alpha*torch.pow(self.direction(x),2).sum(axis=1).sum(axis=-1)

### Zero discount 

In [ ]:
'''Zero drift coefficient for all components'''  
class zero_discount(coefficient):
    def __init__(self,params):
        super(zero_discount, self).__init__(params)
    def __call__(self,x):
        return torch.zeros(x.shape[0],1)

### Terminal condition

In [ ]:
class terminal(coefficient):
    def __init__(self,params):
        self.eta = params['eta']
    def __call__(self,x):
        return (torch.tensor([1.0])-torch.exp(-self.eta*x[:,0])).unsqueeze(-1)



In [ ]:
input = x_tmp[:,0:dim]

In [ ]:
x = sqr_jacobian(test_fnc,input)
sigma = constant_diff(params)
torch.sqrt(torch.pow(params['lb'],2).sum())*test_fnc(input)[:,0]

In [ ]:
x__ = torch.zeros(2,dim,dim)
x__[:,0,0] = -torch.sqrt(torch.pow(params['lb'],2).sum())*torch.abs(test_fnc(input)[:,0])
torch.pow(x__-torch.transpose(sigma(input),1,2),2).sum(axis=-1).sum(axis=-1),*sqr_jacobian(test_fnc,input).shape

In [ ]:
L = ell_source(params,test_fnc,sigma,torch.tensor([1.0]))
L(input)

In [ ]:
D = OU_drift_lin(params,test_fnc,sigma)
D(input)

In [ ]:
x_tmp = torch.tensor([[0.,0.1,-1.4,.18],[0.,0.2,0.1672,.1]])

In [ ]:
diff = NN_diff_1D(params)#x[:,0]=time x[:,1:] space 
diff(input[:,1:]).reshape(input.shape[0],dim,dim)

In [ ]:
g = terminal(params)
g(x_tmp[:,1:])

### Closed-form solution

In [ ]:
class solution(object):
    def __init__(self,params):
        self.dim=params['dim']#2
        self.nu = params['nu']
        self.kappa = params['kappa']
        self.theta = params['theta']
        self.eta = params['eta']
        self.lb = params['lb']
        self.T = params['T']
    def __call__(self,x):
        return 1.-torch.exp(-self.eta*x[:,1]+0.5*torch.sum(torch.pow(self.lb,2))*(self.T-x[:,0])).to(device)


In [ ]:
sol_exp = solution(params0)
sol_exp(input)

#### Plot closed form solution

In [ ]:
steps =100
x = torch.linspace(0.0,1.,steps=steps)
y = torch.linspace(0.0,1.,steps=steps)
xy = torch.cartesian_prod(x,y)
x, y = torch.meshgrid(x, y, indexing='ij')
v_T = g(xy).detach().reshape(steps,steps).squeeze(-1)
v = sol_exp(xy).detach().reshape(steps,steps).squeeze(-1)
f = plt.figure(figsize=(10,3),dpi=300);
ax0 = f.add_subplot(1, 2, 1, projection='3d')
ax0.plot_surface(x.numpy(),y.numpy(),v.numpy(),color='b',alpha=0.5);
ax0.set_xlabel('time')
ax0.set_ylabel('wealth')

ax1 = f.add_subplot(1, 2, 2, projection='3d')
ax1.plot_surface(x.numpy(),y.numpy(),v_T.numpy(),color='r',alpha=0.5);
ax1.set_xlabel('wealth')
ax1.set_ylabel('volatility');
plt.tight_layout();


# Solving linear equations

$\begin{cases}-\partial v_t - \dfrac12 (\sigma^\intercal\sigma): D^2v -\mu\cdot\nabla v + f(t,x) =0\\
v(T,x)=g(x)
\end{cases}$

$\sigma(t,x)$ has shape [num_samples,dim,dim]
$dW_t$  has shape [num_samples,dim]

We need to unsqueeze $dW_t$ before batch multiplication.

    
    torch.bmm(sigma(x[:,:,i]),dw[:, :, i]dw[:, :, i].unsqueeze(2))



In [ ]:
t0 = time.time()
size = [2**10, 9, 9]
iid = torch.randn(size=size).to(device)
print("It takes {:.3E} ms to generate iid samples.".format(round(1000*(time.time()-t0),6)))

In [ ]:
class linear(object):
    def __init__(self,sigma,mu,source,kappa,terminal,T,n,a,b,dim):
        num_samples = 2**10
        self.mu = mu
        self.sigma = sigma 
        self.kappa = kappa 
        self.source = source 
        self.n = n
        self.loss_epoch = []
        self.num_epochs = 10000
        self.epoch=0
        self.x = torch.zeros((num_samples,dim+1,n+1))
        self.Y0 = Ynet()
        self.Z = Znet()
        self.terminal = terminal
        self.dt = torch.tensor([T/n]).to(device)
        dw = torch.randn(size=[num_samples, dim, n]).to(device)* torch.sqrt(self.dt)#you can make randomness a universal variable if 
        self.x[:,1:,0]= a+(b-a)*torch.rand(num_samples,dim).to(device)
        self.r = torch.ones((num_samples,1,n+1)).to(device)
        self.c = torch.zeros((num_samples,1,n+1)).to(device)
        self.sigmadw = torch.zeros((num_samples,dim,n)).to(device)
        for i in range(self.n):
            self.sigmadw[:,:,i] = torch.bmm(self.sigma(self.x[:,:,i]).reshape((num_samples,dim,dim)),dw[:, :, i].unsqueeze(2)).squeeze(2)
            # print(self.x[:,1:,i].shape,self.mu(self.x[:,:,i]).shape,self.sigmadw[:,:,i].shape)
            self.x[:,1:,i+1] = self.x[:,1:,i] + self.mu(self.x[:,:,i])*self.dt + self.sigmadw[:,:,i]
            self.x[:,0,i+1] = self.x[:,0,i]+self.dt
            self.r[:,:,i+1] = self.r[:,:,i]* torch.exp(-self.kappa(self.x[:,:,i])*self.dt)
            if i == self.n -1 :
                self.c[:,:,i+1] = self.terminal(self.x[:,1:,i+1])
            self.c[:,:,i] = self.source(self.x[:,:,i])
        self.x = self.x.clone().detach()
        self.r = self.r.clone().detach()
        self.sigmadw = self.sigmadw.clone().detach()
        self.c = self.c.clone().detach()
        
    def loss(self):
        # self.Zsigmadw = torch.zeros((num_samples,1,n)).to(device)
        for i in range(self.n):   
            if i == 0:
                Y =  self.Y0(self.x[:,1:,0])
            else:
                # self.Zsigmadw[:,:,i] = torch.bmm(self.Z(self.x[:,:,i]).unsqueeze(1),self.sigmadw[:,:,i].unsqueeze(2)).squeeze(2)
                Y = Y*self.r[:,:,i] - self.c[:,:,i]*self.dt + torch.bmm(self.Z(self.x[:,:,i]).unsqueeze(1),self.sigmadw[:,:,i].unsqueeze(2)).squeeze(2)
        return torch.pow(self.c[:,:,-1]-Y,2).mean()
        
    def train(self,lr):
        t_0 = time.time()
        self.lr = lr
        parameters = list(self.Y0.parameters()) + list(self.Z.parameters())
        optimizer = optim.Adam(parameters, self.lr)
        L_ = torch.Tensor([-2.0])
        loss = torch.Tensor([2.0])
        while (torch.abs(L_-loss)>1e-10) & (self.epoch < self.num_epochs):# epoch in range(num_epochs):
            t_1 = time.time()
            optimizer.zero_grad()
            loss= self.loss()#self.cost(self.X,self.modelu(X))+ torch.mean(self.terminal(update(self.X,self.modelu(X))))#
            loss.backward()
            optimizer.step()
            self.loss_epoch.append(loss)
            if self.epoch>0:
                L_ = self.loss_epoch[self.epoch-1]
            if (self.epoch % int(self.num_epochs/10)== int(self.num_epochs/10)-1):
                print("At epoch {} the mean error is {:.2E}.".format(self.epoch+1,loss.detach()))
                self.time_display(t_0, t_1)
            self.epoch += 1
        print("Training took {} epochs.".format(self.epoch))

    def time_display(self, t_0, t_1):
        print("Training this epoch takes {} ms.".format(round(1000*(time.time()-t_1),2)))
        print("So far: {} ms in training.".format(round(1000*(time.time()-t_0),2)))
                
                



In [ ]:
s = constant_diff(params0)
m = zero_drift(params0)
k = zero_discount(params0)
f = zero_source(params0)
g=terminal(params0)
a = torch.tensor([0.0])
b = torch.tensor([1.0])

In [ ]:
num_time_interval = 20
heat = linear(s,m,f,k,g,T,20,a,b,dim)

In [ ]:
heat.train(lr=1e-2)

In [ ]:
plt.plot([l.detach().numpy() for l in heat.loss_epoch]);

In [ ]:
plt.plot(np.log(np.arange(len(heat.loss_epoch))+1),[np.log(l.detach().numpy()) for l in heat.loss_epoch]);

## Plot value function

In [ ]:
def display_it(eqn):
    sol_exp = solution(params0)
    if dim == 2:
        steps =100
        x = torch.linspace(0.0,1.,steps=steps)
        y = torch.linspace(0.0,1.,steps=steps)
        xy = torch.cartesian_prod(x,y)
        txy = torch.cat((torch.zeros(xy.shape[0],1),xy),axis=1)
        v_T = eqn.terminal(xy).detach().reshape(steps,steps).squeeze(-1)
        v = eqn.Y0(xy).detach().reshape(steps,steps).squeeze(-1)
        v_0 = sol_exp(txy).detach().reshape(steps,steps).squeeze(-1)
        f,ax = plt.subplots(1,1,figsize=(15,5),dpi=300);
        ax = plt.axes(projection='3d')
        X, Y = torch.meshgrid(x, y, indexing='ij')
        ax.plot_surface(X.numpy(),Y.numpy(),v.numpy());
        ax.plot_surface(X.numpy(),Y.numpy(),v_0.numpy(),color='r',alpha=0.5);
        ax.set_xlabel('wealth')
        ax.set_ylabel('volatility');
        plt.tight_layout();
    data = torch.rand(2**14,dim)
    v = eqn.Y0(data).detach().squeeze(-1)
    tdata = torch.cat((torch.zeros(data.shape[0],1),data),axis=1)
    v_0 = sol_exp(tdata).detach().squeeze(-1)
    print("The MSE is {:.3E}.".format(pow(v-v_0,2).mean()))  



In [ ]:
display_it(heat);

# Solving semilinear equatoins

In [ ]:
class semilinear(object):
    def __init__(self,sigma,mu,F,kappa,terminal,T,n,a,b,dim):
        self.num_samples = 2**10
        self.mu = mu
        self.sigma = sigma
        self.kappa = kappa 
        self.F = F
        self.n = n
        self.x = torch.zeros((self.num_samples,dim+1,n+1))
        self.Y0 = Ynet()
        self.Z = Znet()
        self.terminal = terminal
        self.epoch=0
        self.loss_epoch = []
        self.num_epochs = 10000
        self.dt = torch.tensor([T/n]).to(device)
        dw = torch.randn(size=[self.num_samples, dim, n]).to(device)* torch.sqrt(self.dt)
        self.x[:,1:,0]= a+(b-a)*torch.rand(self.num_samples,dim).to(device)
        self.r = torch.ones((self.num_samples,1,n+1)).to(device)

        self.sigmadw = torch.zeros((self.num_samples,dim,n)).to(device)
        for i in range(self.n):
            self.sigmadw[:,:,i] = torch.bmm(self.sigma(self.x[:,:,i]).reshape((self.num_samples,dim,dim)),dw[:, :, i].unsqueeze(2)).squeeze(2)
            self.x[:,1:,i+1] = self.x[:,1:,i] + self.mu(self.x[:,:,i])*self.dt + self.sigmadw[:,:,i]
            self.x[:,0,i+1] = self.x[:,0,i]+self.dt
            # print(torch.exp(-self.kappa(self.x[:,:,i])).shape, self.r[:,:,i].shape)
            self.r[:,:,i+1] = self.r[:,:,i]* torch.exp(-self.kappa(self.x[:,:,i])*self.dt)
        self.x = self.x.clone().detach()
        self.sigmadw = self.sigmadw.clone().detach()
        self.r = self.r.clone().detach()
        
    def loss(self):
        c = torch.zeros((self.num_samples,1,self.n+1)).to(device)
        for i in range(self.n):   
            if i == 0:
                Y =  self.Y0(self.x[:,1:,0])
            else:
                if i == self.n - 1:
                    c[:,:,i+1] = self.terminal(self.x[:,1:,i+1])
                c[:,:,i] = self.F(self.x[:,1:,i].unsqueeze(-1),self.Z(self.x[:,:,i]).unsqueeze(-1),self.sigma(self.x[:,:,i]).reshape((self.num_samples,dim,dim))[:,0,0]).unsqueeze(-1)
                Y = Y*self.r[:,:,i] - c[:,:,i]*self.dt + torch.bmm(self.Z(self.x[:,:,i]).unsqueeze(1),self.sigmadw[:,:,i].unsqueeze(2)).squeeze(2)
                # print(Y)
        return torch.pow(c[:,:,-1]-Y,2).mean()
        
    def train(self,lr):
        t_0 = time.time()
        self.lr = lr
        parameters = list(self.Y0.parameters()) + list(self.Z.parameters())
        optimizer = optim.Adam(parameters, self.lr)
        L_ = torch.Tensor([-2.0])
        loss = torch.Tensor([2.0])
        while  (torch.abs(L_-loss)>1e-10) & (self.epoch < self.num_epochs):# 
            t_1 = time.time()
            optimizer.zero_grad()
            loss= self.loss()##
            loss.backward()
            optimizer.step()
            self.loss_epoch.append(loss)
            if self.epoch>0:
                L_ = self.loss_epoch[self.epoch-1]
            if (self.epoch % int(self.num_epochs/10)== int(self.num_epochs/10)-1):
                print("At epoch {} the mean error is {:.2E}.".format(self.epoch+1,loss.detach()))
                self.time_display(t_0,t_1)          
            self.epoch += 1
                
    def time_display(self,t_0,t_1):
        print("Training this epoch takes {:.3E}s.".format(time.time()-t_1))
        print("So far spend {:.3E} in training.".format(time.time()-t_0))      
                



In [ ]:
s = constant_diff(params0)
m = OU_drift_semi(params0)
k = zero_discount(params0)
f = zero_source(params0)
g=terminal(params0)
F = f_driver(params0)
a = torch.tensor([0.0])
b = torch.tensor([1.0])

In [ ]:
semi = semilinear(s,m,F,k,g,params0['T'],20,a,b,dim)

In [ ]:
semi.train(lr=1e-2)

## Plot value function 2D

In [ ]:
display_it(semi)

In [ ]:
plt.plot([l.detach().numpy() for l in semi.loss_epoch]);

In [ ]:
plt.plot(np.log(np.arange(len(semi.loss_epoch))+1),[np.log(l.detach().numpy()) for l in semi.loss_epoch]);

# Combine both linear and semilinear

In [ ]:
class eqn(object):
    def __init__(self,params,sigma,mu,kappa,terminal): #params=dim,T,n,a,b,num_samples
        self.dim = params['dim']
        self.T = params['T']
        self.num_samples = params['num_samples']
        self.n = params['num_steps']
        self.a = params['a']
        self.b = params['b']
        self.loss_epoch = []
        self.num_epochs = 10000
        self.epoch=0
        self.sigma = sigma         
        self.mu = mu
        self.kappa = kappa 
        self.terminal = terminal
        self.x = torch.zeros((self.num_samples,self.dim+1,self.n+1))
        self.dt = torch.tensor([self.T/self.n]).to(device)
        self.Y0 = Ynet()
        self.Z = Znet()
        self.dw = torch.randn(size=[self.num_samples, self.dim, self.n]).to(device)* torch.sqrt(self.dt)
        self.x[:,1:,0]= self.a+(self.b-self.a)*torch.rand(self.num_samples,self.dim).to(device)
        self.r = torch.ones((self.num_samples,1,self.n+1)).to(device)
        self.sigmadw = torch.zeros((self.num_samples,self.dim,self.n)).to(device)
        for i in range(self.n):
            self.sigmadw[:,:,i] = torch.bmm(self.sigma(self.x[:,:,i]).reshape((self.num_samples,self.dim,self.dim)),self.dw[:, :, i].unsqueeze(2)).squeeze(2)
            self.x[:,1:,i+1] = self.x[:,1:,i] + self.mu(self.x[:,:,i])*self.dt + self.sigmadw[:,:,i]
            self.x[:,0,i+1] = self.x[:,0,i]+self.dt
            self.r[:,:,i+1] = self.r[:,:,i]* torch.exp(-self.kappa(self.x[:,:,i])*self.dt)
        self.x = self.x.clone().detach()
        self.r = self.r.clone().detach()
        self.sigmadw = self.sigmadw.clone().detach()
        
        
    def train(self):
        t_0 = time.time()
        self.lr = 1e-2
        parameters = list(self.Y0.parameters()) + list(self.Z.parameters())
        optimizer = optim.Adam(parameters, self.lr)
        L_ = torch.Tensor([-2.0])
        loss = torch.Tensor([2.0])
        while  (torch.abs(L_-loss)>1e-10) & (self.epoch < self.num_epochs):# epoch in range(num_epochs):
            t_1 = time.time()
            optimizer.zero_grad()
            loss= self.loss()##
            loss.backward()
            optimizer.step()
            self.loss_epoch.append(loss)
            if self.epoch>0:
                L_ = self.loss_epoch[self.epoch-1]
            if (self.epoch % int(self.num_epochs/10)== int(self.num_epochs/10)-1):
                print("At epoch {} the mean loss is {:.2E}.".format(self.epoch+1,loss.detach()))
                self.time_display(t_0,t_1)          
            self.epoch += 1
        print("Training took {} epochs.".format(self.epoch))
            
                
    def time_display(self, t_0, t_1):
        print("Training this epoch takes {:.3E} ms.".format(round(1000*(time.time()-t_1),2)))
        print("So far: {:.3E} ms in training.".format(round(1000*(time.time()-t_0),2)))    

class lin(eqn):
    def __init__(self,params,sigma,mu,kappa,terminal,source):
        self.source = source    
        super(lin,self).__init__(params,sigma,mu,kappa,terminal)
        self.c = torch.zeros((self.num_samples,1,self.n+1)).to(device)
        for i in range(self.n):
            if i == self.n -1 :
                self.c[:,:,i+1] = self.terminal(self.x[:,1:,i+1])
            self.c[:,:,i] = self.source(self.x[:,:,i])
            self.c = self.c.clone().detach()

            
    def loss(self):
        # self.Zsigmadw = torch.zeros((num_samples,1,n)).to(device)
        for i in range(self.n):   
            if i == 0:
                Y =  self.Y0(self.x[:,1:,0])
            else:
                # self.Zsigmadw[:,:,i] = torch.bmm(self.Z(self.x[:,:,i]).unsqueeze(1),self.sigmadw[:,:,i].unsqueeze(2)).squeeze(2)
                Y = Y*self.r[:,:,i] - self.c[:,:,i]*self.dt + torch.bmm(self.Z(self.x[:,:,i]).unsqueeze(1),self.sigmadw[:,:,i].unsqueeze(2)).squeeze(2)
                # print(Y)
        return torch.pow(self.c[:,:,-1]-Y,2).mean()
    
class semi(eqn):
    def __init__(self,params,sigma,mu,kappa,terminal,F):
        self.F = F    
        super(semi,self).__init__(params,sigma,mu,kappa,terminal)

            
    def loss(self):
        c = torch.zeros((self.num_samples,1,self.n+1)).to(device)
        for i in range(self.n):   
            if i == 0:
                Y =  self.Y0(self.x[:,1:,0])
            else:
                if i == self.n - 1:
                    c[:,:,i+1] = self.terminal(self.x[:,1:,i+1])
                c[:,:,i] = self.F.eval(self.x[:,1:,i].unsqueeze(-1),self.Z(self.x[:,:,i]).unsqueeze(-1),self.sigma(self.x[:,:,i]).reshape((self.num_samples,dim,dim))[:,0,0]).unsqueeze(-1)
                Y = Y*self.r[:,:,i] - c[:,:,i]*self.dt + torch.bmm(self.Z(self.x[:,:,i]).unsqueeze(1),self.sigmadw[:,:,i].unsqueeze(2)).squeeze(2)
                # print(Y)
        return torch.pow(c[:,:,-1]-Y,2).mean()        

In [ ]:
params = {'dim':2,
        'T':1,
        'num_samples':2**10,
        'num_steps':20,
        'a':torch.tensor([0.0]),
        'b':torch.tensor([1.0])}
x=torch.rand((8,dim+1))
s = sigma()
m = mu()
k = kappa()
f = source()
a = torch.tensor([0.0])
b = torch.tensor([1.0])
num_time_interval = 20

In [ ]:
lin_eqn = lin(params,s,m,k,expo,f)

In [ ]:
lin_eqn.n

In [ ]:
lin_eqn.train()

## Plot value function

In [ ]:
steps =100
x = torch.linspace(0.25,0.75,steps=steps)
y = torch.linspace(0.25,0.75,steps=steps)
xy = torch.cartesian_prod(x,y)
x, y = torch.meshgrid(x, y, indexing='ij')
z_T = lin.terminal(xy).detach().reshape(steps,steps).squeeze(-1)
z = lin.Y0(xy).detach().reshape(steps,steps).squeeze(-1)


In [ ]:
ax = plt.axes(projection='3d')
ax.plot_surface(x.numpy(),y.numpy(),z.numpy());
ax.plot_surface(x.numpy(),y.numpy(),z_T.numpy());


In [ ]:
steps =100
x = torch.linspace(0,1,steps=steps)
y = torch.linspace(0,1,steps=steps)
xy = torch.cartesian_prod(x,y)
x, y = torch.meshgrid(x, y, indexing='ij')
z_T = lin.terminal(xy).detach().reshape(steps,steps).squeeze(-1)
txy = torch.cat((torch.zeros((xy.shape[0],1)),xy),axis=1)
z = lin.Z(txy).detach().reshape(steps,steps,2).squeeze(-1)


In [ ]:
ax = plt.axes(projection='3d')
ax.plot_surface(x.numpy(),y.numpy(),z[:,:,0].numpy());
# ax.plot_surface(x.numpy(),y.numpy(),z_T.numpy());


In [ ]:
ax = plt.axes(projection='3d')
ax.plot_surface(x.numpy(),y.numpy(),z[:,:,1].numpy());
# ax.plot_surface(x.numpy(),y.numpy(),z_T.numpy());


In [ ]:
semi1 = semi(params,s,m,k,expo,F)

In [ ]:
semi1.train()

## Plot value function

In [ ]:
steps =100
x = torch.linspace(0.25,0.75,steps=steps)
y = torch.linspace(0.25,0.75,steps=steps)
xy = torch.cartesian_prod(x,y)
x, y = torch.meshgrid(x, y, indexing='ij')
z_T = semi1.terminal(xy).detach().reshape(steps,steps).squeeze(-1)
z = semi1.Y0(xy).detach().reshape(steps,steps).squeeze(-1)


In [ ]:
ax = plt.axes(projection='3d')
ax.plot_surface(x.numpy(),y.numpy(),z.numpy());
ax.plot_surface(x.numpy(),y.numpy(),z_T.numpy());


In [ ]:
steps =100
x = torch.linspace(0,1,steps=steps)
y = torch.linspace(0,1,steps=steps)
xy = torch.cartesian_prod(x,y)
x, y = torch.meshgrid(x, y, indexing='ij')
z_T = semi1.terminal(xy).detach().reshape(steps,steps).squeeze(-1)
txy = torch.cat((torch.zeros((xy.shape[0],1)),xy),axis=1)
z = semi1.Z(txy).detach().reshape(steps,steps,2).squeeze(-1)


In [ ]:
ax = plt.axes(projection='3d')
ax.plot_surface(x.numpy(),y.numpy(),z[:,:,0].numpy());
# ax.plot_surface(x.numpy(),y.numpy(),z_T.numpy());


In [ ]:
ax = plt.axes(projection='3d')
ax.plot_surface(x.numpy(),y.numpy(),z[:,:,1].numpy());
# ax.plot_surface(x.numpy(),y.numpy(),z_T.numpy());


# Finding second derivative

In [ ]:
import torch
from torch.autograd import grad
 
def test_fn(x):
    return torch.bmm(x.unsqueeze(-1),x.unsqueeze(1)).unsqueeze(-1)

x = torch.tensor([[0.0, 1.0],[1.0,0.0]], requires_grad=True)
 
# Compute the gradient of f with respect to x
def gradiant(x,f):
    return grad(outputs=f(x), inputs=x, grad_outputs=torch.ones_like(f(x)), create_graph=True, retain_graph=True, only_inputs=True)[0]#.unsqueeze(-1)
 
# Print the gradient
print(gradiant(x,test_fn).shape)

In [ ]:
def der1(x,f):  #output= [M,D,1], input: x=[M,D,1]
  Du=torch.zeros(x.shape[0],dim).to(device)
  xin=x.clone().detach()
  xin.requires_grad=True
  # print('xin shape', xin.shape)
  u=f(xin)
  # print(u)
  # print('u/f(xin) shape', u.shape)
  Du=torch.autograd.grad(outputs=[u],inputs=[xin],grad_outputs=torch.ones_like(u),
                          allow_unused=True,retain_graph=True,create_graph=True)[0].unsqueeze(-1)
  # print('Du shape before reshaped', Du.shape)
  Du=torch.reshape(Du,(Du.shape[0],dim,1))
  # print('Du shape after reshaped', Du.shape)
  return Du

In [ ]:
der1(x,test_fn).shape

In [ ]:
def Grad_Hess(x,f_): #output= [M,D,D], #input: x=[M,D], t=[M,1], xt= [M,D+1]
    hess_temp=torch.zeros(x.shape[0],dim,dim).to(device)
    Du=torch.zeros(x.shape[0],dim).to(device)
    xin=x[:,1:].clone().detach()
    xin.requires_grad=True
    u=f_(xin)
    Du=torch.autograd.grad(outputs=[u],inputs=[xin],grad_outputs=torch.ones_like(u),
                           allow_unused=True,retain_graph=True,create_graph=True)[0].unsqueeze(2)
    hess_temp= torch.cat([ torch.autograd.grad(outputs=[Du[:,i,:]],inputs=[xin],grad_outputs=torch.ones_like(Du[:,i,:]),
                           allow_unused=True,retain_graph=True,create_graph=True)[0] for i in range(dim)],1)
    Du = torch.reshape(Du,(Du.shape[0],dim,1))
    # print("Du after reshape:",Du.shape)
    hess_temp=torch.reshape(hess_temp,(hess_temp.shape[0],dim,dim))
    return Du, hess_temp


In [ ]:
semi1.x[:,:,0].squeeze(-1).shape,x.shape

In [ ]:
x = torch.tensor([[0.0,0.0,1.0],[0.0,1.0,0.0],[0.0,1.4,0.7]], requires_grad=True)
Grad_Hess(semi1.x[:,:,0],semi1.Y0)[1].mean()

# test second derivative with NN

<font color="red" size="4">Batch normaizations srews up the derivative.</font>

In [ ]:
def Hess(x,f_): #output= [M,D,D], #input: x=[M,D]
    dim = x.shape[1]
    hess_temp=torch.zeros(x.shape[0],dim,dim).to(device)
    Du=torch.zeros(x.shape[0],dim).to(device)
    xin=x.clone().detach()
    xin.requires_grad=True
    u=f_(xin)
    Du=torch.autograd.grad(outputs=[u],inputs=[xin],grad_outputs=torch.ones_like(u),
                           allow_unused=True,retain_graph=True,create_graph=True)[0].unsqueeze(2)
    hess_temp= torch.cat([ torch.autograd.grad(outputs=[Du[:,i,:]],inputs=[xin],grad_outputs=torch.ones_like(Du[:,i,:]),
                           allow_unused=True,retain_graph=True,create_graph=True)[0] for i in range(dim)],1)
    Du = torch.reshape(Du,(Du.shape[0],dim,1))
    # print("Du after reshape:",Du.shape)
    hess_temp=torch.reshape(hess_temp,(hess_temp.shape[0],dim,dim))
    return Du, hess_temp

In [ ]:
class NN1d(nn.Module): #input [M,D+1]   #output [M,1]
    def __init__(self):
        super(NN1d, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(1, 20),
            # nn.BatchNorm1d(num_features=20),
            nn.Tanh(),
            nn.Linear(20,1),
        )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits#.reshape([dim,dim]) 

In [ ]:
test_fn = NN1d()

In [ ]:
x_tr = torch.rand(100,1)
# x_tr = torch.rand(0,1,100).unsqueeze(-1)
y_tr = x_tr*x_tr

In [ ]:
num_epochs = 100000
lr = 1e-2
optimizer = optim.Adam(test_fn.parameters(), lr)
L_ = torch.Tensor([-2.0])
loss = torch.Tensor([2.0])
epoch=0
while  (torch.abs(L_-loss)>1e-8) & (epoch < num_epochs):# epoch in range(num_epochs):
    optimizer.zero_grad()
    loss= torch.pow(y_tr-test_fn(x_tr),2).mean()##
    loss.backward()
    optimizer.step()
    if (epoch % int(num_epochs/10)== int(num_epochs/10)-1):
        print("At epoch {} the mean loss is {}.".format(epoch+1,loss.detach()))
    epoch += 1

In [ ]:
x_ts = torch.linspace(0,1,100).unsqueeze(-1)
plt.plot(x_ts.detach().numpy(),test_fn(x_ts).detach().numpy());
plt.plot(x_ts.detach().numpy(),x_ts*x_ts.detach().numpy());

In [ ]:
xin = x_ts.clone().detach()
xin.requires_grad=True
dy = gradiant(xin,test_fn).squeeze(-1)
dy.shape

In [ ]:
# plt.plot(x_tr.detach().numpy(),test_fn(x_tr).detach().numpy());
plt.plot(x_ts.detach().numpy(),dy.detach().numpy());

In [ ]:
gradiant(torch.tensor([[1.0],[0.5],[0.0]],requires_grad=True),test_fn)

In [ ]:
ddy = Hess(x_ts,test_fn)[1].squeeze(-1)
plt.plot(x_ts.detach().numpy(),ddy.detach().numpy());

${\color{red}\text{It does not matter if the second derivative is not accurate. } \ell^2  \text{ will be positive and the direction is always a direction of increase.}}$